## Airbnb project

In [196]:
import pandas as pd
import geopandas as gpd
import geoplot
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import seaborn as sns
import numpy as np
import shapely
from geopy import distance

In [197]:
pd.options.mode.chained_assignment = None

In [198]:
#hood = gpd.read_file('neighbourhoods.geojson')
metro = gpd.read_file('metro.geojson')
df = pd.read_csv('listings.csv')

# Metro dataset

In [199]:
df2 = metro.copy()
for i in range(len(df2)):
    if not '/' in df2['LINHA'][i]:
        df2.drop([i], axis=0, inplace=True)
df2 = df2.reset_index()

In [200]:
for i in range(len(metro)):
    if '/' in  metro['LINHA'][i]:
        metro['LINHA'][i] = metro['LINHA'][i].split('/')[0]

for j in range(len(df2)):
    df2['LINHA'][j] = df2['LINHA'][j].split('/')[1]
    
metro = pd.concat([metro, df2])

In [201]:
metro = pd.concat([metro, df2])
metro = metro.drop(columns = ['index','OBJECTID','COD_SIG','IDTIPO','SITUACAO','GlobalID'])

In [202]:
metro

,NOME,LINHA,geometry
0,Cais do Sodré,Verde,POINT (-9.14609 38.70627)
1,Terreiro do Paço,Azul,POINT (-9.13419 38.70728)
2,Baixa Chiado,Azul,POINT (-9.14015 38.71057)
3,Santa Apolónia,Azul,POINT (-9.12241 38.71404)
4,Rossio,Verde,POINT (-9.13792 38.71418)
...,...,...,...
1,Marquês de Pombal,Amarela,POINT (-9.15005 38.72532)
2,São Sebastião,Vermelha,POINT (-9.15394 38.73453)
3,Saldanha,Vermelha,POINT (-9.14533 38.73532)
4,Alameda,Vermelha,POINT (-9.13387 38.73697)


In [203]:
yellow = metro[metro.LINHA == 'Amarela'].reset_index()
blue = metro[metro.LINHA == 'Azul'].reset_index()
green = metro[metro.LINHA == 'Verde'].reset_index()
red = metro[metro.LINHA == 'Vermelha'].reset_index()

# Metro Network

In [204]:
import networkx as nx
from pyvis.network import Network

Import the graph baseline from excel where we have column for Origin and Destination

In [205]:
metro_graph = pd.read_excel('metro.xlsx')

In [206]:
metro_graph = metro_graph.dropna()
metro_graph = metro_graph.drop(columns = ['NOME', 'LINHA', 'Unnamed: 0', 'OBJECTID'])

## Set up the graph

In [207]:
G = nx.Graph()
G = nx.from_pandas_edgelist(metro_graph, 'Origin', 'Destination')

In [208]:
net = Network(notebook=True)
net.from_nx(G)
net.width=500
net.height=500
net.show('metro.html')

Get the origin and destination coordinates

In [209]:
metro_dict = dict(zip(metro.NOME, metro.geometry))
metro_graph['p1'] = (metro_graph['Origin']).map(metro_dict)
metro_graph['p2'] = (metro_graph['Destination']).map(metro_dict)

Create a LineString object between origin and Destination

In [210]:
a = []
for i in range(len(metro_graph)):
    a.append((LineString([metro_graph.p1[i],metro_graph.p2[i]])))

metro_graph['LineString'] = a

Transform the graph in a Geopandas Datframe

In [211]:
metro_graph = gpd.GeoDataFrame(metro_graph)

## Calculate distance between nodes

In [212]:
from shapely.geometry import LineString
from shapely.ops import transform
from functools import partial
import pyproj

This function transforms the LineString distance in Meters

In [213]:
def transform_linestring(df):
    
    lenghts = []
    project = partial(pyproj.transform, pyproj.Proj('EPSG:4326'), pyproj.Proj('EPSG:32633'))
    for line in df.LineString:
        lenghts.append(int(transform(project, line).length))
    
    df['distance_meters'] = lenghts

In [214]:
transform_linestring(metro_graph)

We now have a graph with a distance attribute to calculate degree importance based on.

In [215]:
G_distance = nx.from_pandas_edgelist(metro_graph, 'Origin', 'Destination',['distance_meters'])

### Degrees

In [216]:
#degree_sort = sorted(G_distance.degree(), key=lambda x: x[1], reverse=True)
degrees = {node:val for (node, val) in G_distance.degree()}

closeness = nx.closeness_centrality(G_distance, distance='distance_meters')

between = nx.betweenness_centrality(G_distance, weight='distance_meters')

eigen = nx.eigenvector_centrality(G_distance,max_iter=10000, weight='distance_meters')

centrality = nx.degree_centrality(G_distance)

In [217]:
betweness = pd.DataFrame.from_dict(between, orient='index').rename(columns = {0: 'Betweness'}).reset_index().rename(columns = {'index': 'station'})
closeness = pd.DataFrame.from_dict(closeness, orient='index').rename(columns = {0: 'Closeness'}).reset_index().rename(columns = {'index': 'station'})
eigeness = pd.DataFrame.from_dict(eigen, orient='index').rename(columns = {0: 'Eigeness'}).reset_index().rename(columns = {'index': 'station'})
central = pd.DataFrame.from_dict(centrality, orient='index').rename(columns = {0: 'Centrality'}).reset_index().rename(columns = {'index': 'station'})
Degree = pd.DataFrame.from_dict(degrees, orient='index').rename(columns = {0: 'Degree'}).reset_index().rename(columns = {'index': 'station'})
weight = pd.merge(betweness, closeness, on='station')
weight = pd.merge(weight, eigeness, on='station')
weight = pd.merge(weight, central, on='station')
weight = pd.merge(weight, Degree, on='station')

In [218]:
weight['degree_mean'] = weight.iloc[:,1:4].mean(axis=1)

In [219]:
lines_dict = dict(zip(metro.NOME, metro.LINHA))
station_coord = dict(zip(metro.NOME, metro.geometry))

weight['line_color'] = (weight['station']).map(lines_dict)
weight['Coordinate'] = (weight['station']).map(station_coord)

## Function that gives the distance in meters from a specified point to each metro station

The idea is to have an interactive weight attribution where we are the ones choosing the point of interest, giving coordinates which can be found on this website: https://www.latlong.net/place/rossio-square-lisbon-portugal-23131.html.
Then we assign a score to that monument and depending on how near the metro station is the more importance it gets.

In [220]:
def distance_grade(longlat_tuple, grade, df, name_of_place):
    
    col_name = 'Distance from ' + str(name_of_place)
    
    from_place_list = [] #list of distances
    location = Point(longlat_tuple) #get location of the monument
    project = partial(pyproj.transform, pyproj.Proj('EPSG:4326'), pyproj.Proj('EPSG:32633'))
    for i in df.Coordinate: #iterate over the coordinates of each metro station
        line_string = shapely.geometry.LineString([i,location]) #construct a linestring between the station and the monument
        trans = transform(project, line_string) #transform the linestring in meters
        #from_place_list.append(int(trans.length))
        
    #further implementation to assign weight (importance) to each monument
        if int(trans.length) < 2000: #if the distance is less than 4 km from the monument
            from_place_list.append(grade) 
        elif int(trans.length) < 4000 and int(trans.length) > 2000:
            from_place_list.append(grade*0.5)
        elif int(trans.length) < 8000 and int(trans.length) > 4000:
            from_place_list.append(grade*0.25)
        else:
            from_place_list.append(0)
            
    df[col_name] = from_place_list

## Implement for a list of monuments

In [221]:
list_of_monuments = {'Mosteiro dos Jerónimos': ((-9.204487, 38.697819), 5.5),
                    'Oceanário de Lisboa': ((-9.0937, 38.7635), 6),
                    'Museu Nacional de Arte Antiga': ((-9.141161, 38.708460), 4),
                    'Torre de Belem': ((-9.2147, 38.6967), 8),
                    'Elevador de Santa Justa': ((-9.139221, 38.712173), 9),
                    'Padrão dos Descobrimentos': ((-9.205712, 38.693596), 6),
                    'Praça do Comércio': ((-9.136744, 38.707779), 10),
                    'Museu de Arte, Arquitectura e Tecnologia': ((-9.194453, 38.695927), 6),
                    }

In [222]:
for i in list_of_monuments:
    distance_grade(list_of_monuments[i][0], list_of_monuments[i][1], weight, i)

In [223]:
weight['Monument_Score'] = weight.iloc[:,9:].mean(axis=1).round(2)

## Delete columns in excess

In [228]:
weight = weight.drop(columns = weight.iloc[:,9:-1].columns.tolist())
weight = weight.drop(columns = weight.iloc[:,1:4].columns.tolist())

In [229]:
weight

,station,Centrality,Degree,degree_mean,line_color,Coordinate,Monument_Score
0,Reboleira,0.020408,1,0.000030,Azul,POINT (-9.223883295575069 38.752271287954),0.800
1,Amadora Este,0.040816,2,0.013646,Azul,POINT (-9.217901617117381 38.758715859802),0.610
2,Alfornelos,0.040816,2,0.026693,Azul,POINT (-9.204477478277459 38.7605071115253),0.610
3,Pontinha,0.040816,2,0.039180,Azul,POINT (-9.196937374617089 38.7623917163549),0.360
4,Carnide,0.040816,2,0.051157,Azul,POINT (-9.19273248473962 38.7591726199562),0.610
5,Colégio Militar,0.040816,2,0.062748,Azul,POINT (-9.18982582843268 38.7538737032722),1.080
6,Alto dos Moinhos,0.040816,2,0.073904,Azul,POINT (-9.180553683073009 38.7501358045407),1.520
7,Laranjeiras,0.040816,2,0.084964,Azul,POINT (-9.17248892825242 38.7486544474615),1.270
8,Jardim Zoológico,0.040816,2,0.096579,Azul,POINT (-9.168327471918969 38.7413650553712),1.520
9,Praça de Espanha,0.040816,2,0.109039,Azul,POINT (-9.159328880631289 38.7378628556453),1.550


## Score increase based on Degree

If the metro station has more than degree 2 i.e. it's a crossing of 2 lines: the score is increased by 50%.

In [230]:
for i in range(len(weight)):
    if weight.Degree[i] > 2:
        weight.Monument_Score[i] *= 1.5

In [232]:
weight

,station,Centrality,Degree,degree_mean,line_color,Coordinate,Monument_Score
0,Reboleira,0.020408,1,0.000030,Azul,POINT (-9.223883295575069 38.752271287954),0.8000
1,Amadora Este,0.040816,2,0.013646,Azul,POINT (-9.217901617117381 38.758715859802),0.6100
2,Alfornelos,0.040816,2,0.026693,Azul,POINT (-9.204477478277459 38.7605071115253),0.6100
3,Pontinha,0.040816,2,0.039180,Azul,POINT (-9.196937374617089 38.7623917163549),0.3600
4,Carnide,0.040816,2,0.051157,Azul,POINT (-9.19273248473962 38.7591726199562),0.6100
5,Colégio Militar,0.040816,2,0.062748,Azul,POINT (-9.18982582843268 38.7538737032722),1.0800
6,Alto dos Moinhos,0.040816,2,0.073904,Azul,POINT (-9.180553683073009 38.7501358045407),1.5200
7,Laranjeiras,0.040816,2,0.084964,Azul,POINT (-9.17248892825242 38.7486544474615),1.2700
8,Jardim Zoológico,0.040816,2,0.096579,Azul,POINT (-9.168327471918969 38.7413650553712),1.5200
9,Praça de Espanha,0.040816,2,0.109039,Azul,POINT (-9.159328880631289 38.7378628556453),1.5500
